<a href="https://colab.research.google.com/github/Marcia-Branquinho-Garcia/Agentes-de-IA/blob/main/C%C3%B3pia_de_Assistente_Conteudo_Multimodal_Imersao_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Parte 1: Configuração Inicial
%pip install -q google-genai google-adk requests python-dotenv

# Configurar a API Key do Google Gemini
import os
from google.colab import userdata

# Configure sua API key aqui
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Importar bibliotecas necessárias
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

import re
import json
import random
import textwrap
from datetime import date
from IPython.display import display, Markdown, HTML
import warnings
import requests

warnings.filterwarnings("ignore")

# Inicializar o cliente do Google Genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Função auxiliar para chamar agentes
def call_agent(agent: Agent, message_text: str) -> str:
    """Chama um agente e retorna sua resposta como texto"""
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado
def to_markdown(text):
    """Exibe texto formatado em markdown"""
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("✅ Configuração inicial concluída com sucesso!")


✅ Configuração inicial concluída com sucesso!


In [ ]:
# Parte 2: Agentes de Ideação e Texto

# Agente de Ideação
class IdeacaoAgent:
    def __init__(self):
        self.agent = Agent(
            name="agente_ideacao",
            model=MODEL_ID,
            description="Agente que gera ideias criativas para conteúdo cristão",
            tools=[google_search],
            instruction="""
            Você é um especialista em conteúdo cristão/bíblico e geração de ideias criativas.

            Sua tarefa é analisar o tema bíblico fornecido e gerar ideias originais e
            envolventes que possam ser desenvolvidas em vídeos para um canal cristão no YouTube.

            Para cada ideia gerada:
            1. Forneça um título atraente e inspirador
            2. Apresente uma breve descrição (2-3 frases)
            3. Explique por que essa abordagem seria eficaz para o público cristão

            Use a ferramenta de busca para pesquisar interpretações bíblicas,
            comentários teológicos e exemplos de conteúdo cristão bem-sucedido.

            Suas sugestões devem ser fiéis ao texto bíblico, mas também relevantes
            para a vida contemporânea, especialmente para mulheres cristãs.

            Numere cada ideia claramente (1., 2., etc.) e separe-as por linhas em branco.

            IMPORTANTE: Todo o conteúdo deve ser em português brasileiro, adequado para
            um público cristão contemporâneo.
            """
        )

    def gerar_ideias(self, tema, plataforma, quantidade=5):
        """Gera ideias para conteúdo baseado em tema bíblico"""
        print(f"🧠 Gerando ideias para '{tema}' na plataforma '{plataforma}'...")

        entrada = f"""
        Tema bíblico: {tema}
        Plataforma: {plataforma}
        Quantidade de ideias: {quantidade}

        Gere ideias criativas para vídeos cristãos sobre este tema,
        relacionados à Bíblia e à fé cristã. O conteúdo será para
        um canal cristão no YouTube, voltado para adultos.

        Por favor, gere apenas em português brasileiro.
        """

        ideias = call_agent(self.agent, entrada)
        return ideias


# Agente de Geração de Texto
class TextoAgent:
    def __init__(self):
        self.agent = Agent(
            name="agente_texto",
            model=MODEL_ID,
            description="Agente que gera conteúdo textual para vídeos cristãos",
            tools=[google_search],
            instruction="""
            Você é um redator especializado em conteúdo cristão/bíblico.

            Sua tarefa é desenvolver roteiros e textos para vídeos cristãos no YouTube,
            com base na ideia fornecida.

            O texto deve ser:
            1. Fiel às Escrituras e teologicamente sólido
            2. Bem estruturado, com introdução, desenvolvimento por tópicos e conclusão
            3. Envolvente e inspirador para o público cristão
            4. Inclua referências bíblicas específicas (capítulo e versículo)

            Use a ferramenta de busca para pesquisar versículos bíblicos, comentários
            teológicos e informações contextuais que enriqueçam o conteúdo.

            Utilize formatação em markdown com títulos (# e ##) para estruturar o texto
            em seções que possam ser facilmente convertidas em timestamps para o vídeo.

            IMPORTANTE: Todo o conteúdo deve ser em português brasileiro, em um tom
            acolhedor e inspirador adequado para um público cristão.
            """
        )

    def gerar_texto(self, ideia, tom, tamanho, plataforma):
        """Gera texto completo baseado na ideia selecionada"""
        print(f"📝 Gerando texto com tom '{tom}' e tamanho '{tamanho}'...")

        # Mapear tamanhos para contagens aproximadas de palavras
        tamanhos = {
            "curto": "600-800 palavras (5-7 minutos de vídeo)",
            "médio": "1000-1500 palavras (8-12 minutos de vídeo)",
            "longo": "2000-2500 palavras (15-20 minutos de vídeo)"
        }

        palavra_count = tamanhos.get(tamanho.lower(), tamanhos["médio"])

        entrada = f"""
        Ideia para vídeo cristão: {ideia}
        Tom: {tom}
        Tamanho: {palavra_count}
        Plataforma: {plataforma}

        Desenvolva um roteiro completo para um vídeo cristão no YouTube,
        baseado na ideia acima. Inclua:

        1. Um título envolvente e cristão
        2. Uma introdução que conecte com o público e apresente o tema
        3. Desenvolvimento por tópicos com referências bíblicas específicas
        4. Aplicações práticas para a vida cristã contemporânea
        5. Uma conclusão inspiradora com chamada para ação

        Use formatação markdown com # para título principal e ## para seções
        que servirão como timestamps no vídeo.

        Inclua também uma breve oração relacionada ao tema no final.
        """

        texto = call_agent(self.agent, entrada)
        return texto

    def extrair_palavras_chave(self, texto, quantidade=5):
        """Extrai palavras-chave do texto para SEO"""
        palavras_chave_agent = Agent(
            name="agente_palavras_chave",
            model=MODEL_ID,
            description="Agente que extrai palavras-chave para SEO cristão",
            instruction=f"""
            Extraia as {quantidade} palavras-chave mais relevantes do texto cristão fornecido.
            Essas palavras-chave devem ser relevantes para SEO de conteúdo cristão no YouTube.

            Inclua termos bíblicos, conceitos teológicos e expressões que pessoas cristãs
            buscariam no YouTube relacionadas ao tema.

            Retorne apenas as palavras-chave, uma por linha, sem numeração ou outros textos.
            As palavras-chave devem ser em português.
            """
        )

        # Limitar o texto para evitar problemas com tamanho
        texto_resumido = texto[:3000] + "..." if len(texto) > 3000 else texto

        palavras = call_agent(palavras_chave_agent, texto_resumido)

        # Extrair apenas as palavras-chave, uma por linha
        keywords = [line.strip() for line in palavras.split('\n') if line.strip()]

        # Limitar ao número solicitado
        return keywords[:quantidade]

print("✅ Agentes de Ideação e Texto configurados com sucesso!")


✅ Agentes de Ideação e Texto configurados com sucesso!


In [ ]:
# Parte 3: Agente de Edição

class EditorAgent:
    def __init__(self):
        self.agent = Agent(
            name="agente_editor",
            model=MODEL_ID,
            description="Agente que edita e otimiza conteúdo para YouTube cristão",
            tools=[google_search],
            instruction="""
            Você é um editor profissional especializado em conteúdo cristão para YouTube.

            Sua tarefa é revisar, editar e aprimorar o roteiro do vídeo, considerando:
            1. Clareza e coerência da mensagem bíblica
            2. Estrutura e fluidez da narrativa para um vídeo
            3. Otimização para SEO no YouTube (se palavras-chave fornecidas)
            4. Correção teológica e fidelidade às Escrituras

            Use a ferramenta de busca para pesquisar versículos bíblicos,
            comentários teológicos e melhores práticas para vídeos cristãos no YouTube.

            Mantenha a essência da mensagem original, mas melhore a apresentação,
            ritmo e impacto para um vídeo envolvente.

            Formate o texto final em markdown, mantendo a estrutura de títulos para timestamps.

            Também crie uma descrição otimizada para o vídeo no YouTube e
            sugestões de tags relevantes para o público cristão.
            """
        )

    def otimizar(self, texto, plataforma, palavras_chave=None):
        """Edita e otimiza o conteúdo para o YouTube"""
        print(f"✂️ Editando e otimizando o conteúdo para '{plataforma}'...")

        info_seo = ""
        if palavras_chave and len(palavras_chave) > 0:
            info_seo = f"Palavras-chave para SEO cristão: {', '.join(palavras_chave)}"

        entrada = f"""
        Roteiro do vídeo a ser otimizado:
        {texto}

        Plataforma: {plataforma}

        {info_seo}

        Revise e otimize este roteiro para um vídeo cristão no YouTube.

        1. Mantenha a estrutura de títulos (# e ##) para timestamps
        2. Verifique as referências bíblicas e corrija se necessário
        3. Melhore a introdução para captar atenção nos primeiros 15 segundos
        4. Otimize para SEO usando as palavras-chave fornecidas
        5. Crie uma descrição atraente para o vídeo (até 2000 caracteres)
        6. Sugira 10 tags relevantes para o YouTube

        Mantenha o conteúdo teologicamente sólido e fiel à mensagem bíblica.
        """

        texto_otimizado = call_agent(self.agent, entrada)

        # Formatar o conteúdo final para o YouTube
        conteudo_formatado = self._formatar_youtube(texto_otimizado)

        return conteudo_formatado

    def _formatar_youtube(self, texto):
        """Processa o texto otimizado para extrair informações importantes para YouTube"""
        linhas = texto.split('\n')
        titulo = ""
        descricao = ""
        roteiro = texto
        topicos = []
        timestamps = []
        tags = []

        # Estado para capturar diferentes seções
        capturando_descricao = False
        capturando_tags = False
        tempo_atual = "00:00"

        # Processar linha por linha
        for linha in linhas:
            # Capturar título principal
            if linha.startswith("# ") and not titulo:
                titulo = linha[2:]

            # Capturar subtítulos para timestamps
            elif linha.startswith("## "):
                subtitulo = linha[3:]
                topicos.append(subtitulo)
                timestamps.append(f"{tempo_atual} - {subtitulo}")

                # Simular incremento de tempo
                minutos, segundos = map(int, tempo_atual.split(':'))
                minutos += 2  # Adiciona 2 minutos por seção
                if minutos >= 60:
                    minutos %= 60
                tempo_atual = f"{minutos:02d}:{segundos:02d}"

            # Capturar descrição
            elif "descrição" in linha.lower() or "description" in linha.lower():
                capturando_descricao = True
                capturando_tags = False
                continue

            # Capturar tags
            elif "tags" in linha.lower() or "hashtags" in linha.lower():
                capturando_descricao = False
                capturando_tags = True
                continue

            # Processar linha com base no estado
            if capturando_descricao and linha.strip() and not linha.startswith("#"):
                descricao += linha + "\n"
            elif capturando_tags and linha.strip() and not linha.startswith("#"):
                # Processar linha de tags
                if "," in linha:
                    # Tags separadas por vírgula
                    tags.extend([tag.strip() for tag in linha.split(",") if tag.strip()])
                elif "#" in linha:
                    # Tags com hashtag
                    tags.extend([tag.strip() for tag in linha.split() if tag.strip().startswith("#")])
                else:
                    # Tags separadas por espaço
                    tags.extend([tag.strip() for tag in linha.split() if tag.strip()])

        # Se não encontrou descrição explícita, usar o início do texto
        if not descricao:
            for linha in linhas:
                if not linha.startswith("#") and linha.strip():
                    descricao += linha + "\n"
                    if len(descricao) > 500:
                        break

        # Adicionar informações padrão à descrição
        descricao += "\n\n===================================\n"
        descricao += "🔔 INSCREVA-SE NO CANAL: [URL DO CANAL]\n"
        descricao += "👍 DEIXE SEU LIKE e COMPARTILHE\n"
        descricao += "💬 COMENTE sua opinião ou dúvidas abaixo\n\n"

        # Adicionar timestamps na descrição
        if timestamps:
            descricao += "⏱️ TIMESTAMPS:\n"
            for ts in timestamps:
                descricao += ts + "\n"

        # Se não encontrou tags suficientes, adicionar algumas padrões para conteúdo cristão
        if len(tags) < 5:
            tags_padrao = ["Bíblia", "Provérbios 31", "mulher virtuosa", "cristianismo",
                          "devocional", "estudo bíblico", "mulher cristã", "versículos"]
            tags.extend(tags_padrao)

        # Limitar a 10 tags
        tags = tags[:10]

        # Criar o objeto formatado
        return {
            "tipo": "youtube",
            "titulo": titulo,
            "descricao": descricao,
            "tags": tags,
            "timestamps": timestamps,
            "roteiro": roteiro,
            "info_adicional": "Conteúdo para YouTube sobre a Mulher de Provérbios 31"
        }

print("✅ Agente de Edição configurado com sucesso!")


✅ Agente de Edição configurado com sucesso!


In [ ]:
# Parte 4: Assistente Principal Simplificado (sem imagens)

class AssistenteConteudoSimplificado:
    def __init__(self):
        self.ideacao = IdeacaoAgent()
        self.texto = TextoAgent()
        self.editor = EditorAgent()

        # Armazenar resultados intermediários
        self.ideias = ""
        self.ideia_selecionada = ""
        self.texto_gerado = ""
        self.palavras_chave = []
        self.conteudo_final = {}

    def iniciar(self):
        """Inicia o processo simplificado (sem imagens)"""
        print("🚀 Assistente de Criação de Conteúdo para YouTube Cristão 🚀")
        print("===========================================================")

        try:
            # 1. Obter tema
            tema = input("\n📝 Digite o tema bíblico para o conteúdo (ex: A Mulher de Provérbios 31): ")
            if not tema:
                tema = "A Mulher de Provérbios 31: virtudes e lições para a mulher cristã contemporânea"
                print(f"Usando tema padrão: {tema}")

            # 2. Gerar ideias
            print("\n✨ Etapa 1: Gerando ideias criativas...\n")
            self.ideias = self.ideacao.gerar_ideias(tema, "youtube", 5)
            display(to_markdown(self.ideias))

            # 3. Selecionar uma ideia
            print("\n🔍 Digite o número da ideia que você escolheu (ex: 1, 2, 3...):")

            # Criar uma ideia padrão para fallback
            ideia_padrao = f"Conteúdo sobre {tema} para YouTube com foco cristão"

            # Extrair ideias numeradas
            ideias_extraidas = []
            linhas = self.ideias.split('\n')
            for i, linha in enumerate(linhas):
                if re.match(r'^\d+\.', linha.strip()):
                    resumo = linha.strip()
                    print(f"  Ideia {resumo[:100]}..." if len(resumo) > 100 else f"  {resumo}")
                    ideias_extraidas.append(resumo)

            # Se não encontrou ideias, usar o texto completo
            if not ideias_extraidas:
                ideias_extraidas = [self.ideias]
                print(f"  1. Conteúdo baseado nas sugestões geradas")

            # Selecionar ideia
            try:
                numero_ideia = int(input("> ")) - 1
                if 0 <= numero_ideia < len(ideias_extraidas):
                    self.ideia_selecionada = ideias_extraidas[numero_ideia]
                else:
                    print("⚠️ Número inválido, usando a primeira ideia.")
                    self.ideia_selecionada = ideias_extraidas[0] if ideias_extraidas else ideia_padrao
            except:
                print("⚠️ Entrada inválida, usando a primeira ideia.")
                self.ideia_selecionada = ideias_extraidas[0] if ideias_extraidas else ideia_padrao

            print(f"\n✅ Ideia selecionada: {self.ideia_selecionada[:100]}..."
                  if len(self.ideia_selecionada) > 100 else f"\n✅ Ideia selecionada: {self.ideia_selecionada}")

            # 4. Configurações para geração de texto
            print("\n📋 Configurando a geração de texto...")

            tons = ["inspirador", "informativo", "pastoral", "evangelístico", "devocional"]
            print("Tons disponíveis:")
            for i, t in enumerate(tons):
                print(f"  {i+1}. {t.capitalize()}")

            try:
                tom_idx = int(input("Escolha o número do tom (1-5): ")) - 1
                tom = tons[tom_idx] if 0 <= tom_idx < len(tons) else "inspirador"
            except:
                print("⚠️ Valor inválido, usando tom Inspirador como padrão.")
                tom = "inspirador"

            tamanhos = ["curto", "médio", "longo"]
            print("Tamanhos disponíveis:")
            for i, t in enumerate(tamanhos):
                print(f"  {i+1}. {t.capitalize()}")

            try:
                tamanho_idx = int(input("Escolha o número do tamanho (1-3): ")) - 1
                tamanho = tamanhos[tamanho_idx] if 0 <= tamanho_idx < len(tamanhos) else "médio"
            except:
                print("⚠️ Valor inválido, usando tamanho Médio como padrão.")
                tamanho = "médio"

            # 5. Gerar texto
            print("\n✨ Etapa 2: Gerando texto para a ideia selecionada...\n")
            self.texto_gerado = self.texto.gerar_texto(self.ideia_selecionada, tom, tamanho, "youtube")
            display(to_markdown(self.texto_gerado))

            # 6. Extrair palavras-chave
            print("\n🔑 Extraindo palavras-chave para SEO...")
            self.palavras_chave = self.texto.extrair_palavras_chave(self.texto_gerado, 8)
            print("Palavras-chave extraídas:")
            for kw in self.palavras_chave:
                print(f"  • {kw}")

            # 7. Edição e otimização (sem imagens)
            print("\n✨ Etapa 3: Editando e otimizando o conteúdo...\n")

            # Opcional: palavras-chave adicionais para SEO
            usar_seo = input("Deseja adicionar mais palavras-chave para SEO? (s/n): ").lower() == 's'
            if usar_seo:
                keywords_input = input("Digite palavras-chave adicionais (separadas por vírgula): ")
                if keywords_input:
                    keywords_adicionais = [k.strip() for k in keywords_input.split(',')]
                    self.palavras_chave.extend(keywords_adicionais)
                    # Limitar a 10 palavras-chave
                    self.palavras_chave = self.palavras_chave[:10]

            # Otimizar conteúdo
            self.conteudo_final = self.editor.otimizar(
                self.texto_gerado,
                "youtube",
                self.palavras_chave
            )

            # 8. Exibir resultado final
            self._exibir_conteudo_final()

            # 9. Salvar o conteúdo
            self._salvar_conteudo()

        except Exception as e:
            print(f"⚠️ Ocorreu um erro: {e}")
            import traceback
            traceback.print_exc()

    def _exibir_conteudo_final(self):
        """Exibe o conteúdo final formatado"""
        try:
            print("\n🎉 Conteúdo Final para YouTube 🎉")
            print("===============================")

            titulo = self.conteudo_final.get("titulo", "")
            descricao = self.conteudo_final.get("descricao", "")
            tags = self.conteudo_final.get("tags", [])
            timestamps = self.conteudo_final.get("timestamps", [])
            roteiro = self.conteudo_final.get("roteiro", "")

            print(f"\n📹 Título do Vídeo: {titulo}")

            print("\n⏱️ Timestamps:")
            for ts in timestamps:
                print(f"  • {ts}")

            print("\n🏷️ Tags:")
            print(f"  {', '.join(tags)}")

            print("\n📝 Descrição:")
            print("-" * 50)
            print(descricao[:500] + "..." if len(descricao) > 500 else descricao)
            print("-" * 50)

            print("\n📜 Roteiro do Vídeo:")
            display(to_markdown(roteiro))

        except Exception as e:
            print(f"⚠️ Erro ao exibir conteúdo final: {e}")

    def _salvar_conteudo(self):
        """Salva o conteúdo final em arquivos de texto"""
        salvar = input("\nDeseja salvar o conteúdo gerado? (s/n): ").lower() == 's'

        if salvar:
            nome_base = input("Digite um nome base para os arquivos (ex: mulher_proverbios31): ")
            if not nome_base:
                nome_base = "conteudo_cristao"

            # Salvar roteiro em formato markdown
            with open(f"{nome_base}_roteiro.md", "w", encoding="utf-8") as f:
                f.write(self.conteudo_final.get("roteiro", ""))

            # Salvar descrição e metadados em arquivo separado
            with open(f"{nome_base}_metadados.txt", "w", encoding="utf-8") as f:
                f.write(f"TÍTULO: {self.conteudo_final.get('titulo', '')}\n\n")
                f.write(f"DESCRIÇÃO:\n{self.conteudo_final.get('descricao', '')}\n\n")
                f.write(f"TAGS:\n{', '.join(self.conteudo_final.get('tags', []))}\n\n")
                f.write(f"TIMESTAMPS:\n")
                for ts in self.conteudo_final.get('timestamps', []):
                    f.write(f"{ts}\n")

            print(f"\n✅ Conteúdo salvo como '{nome_base}_roteiro.md' e '{nome_base}_metadados.txt'")

print("✅ Assistente Principal configurado com sucesso!")


✅ Assistente Principal configurado com sucesso!


In [ ]:
# Parte 5: Iniciar o Assistente

# Criar e iniciar o assistente
assistente = AssistenteConteudoSimplificado()
assistente.iniciar()


🚀 Assistente de Criação de Conteúdo para YouTube Cristão 🚀

📝 Digite o tema bíblico para o conteúdo (ex: A Mulher de Provérbios 31): A Mulher de Provérbios 31: virtudes e lições de fé para a mulher cristã contemporânea

✨ Etapa 1: Gerando ideias criativas...

🧠 Gerando ideias para 'A Mulher de Provérbios 31: virtudes e lições de fé para a mulher cristã contemporânea' na plataforma 'youtube'...


> Aqui estão 5 ideias de vídeos criativos para o seu canal cristão no YouTube, inspiradas em Provérbios 31:
> 
>  1. **Título:** Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa
>  * **Descrição:** Aceite o desafio de incorporar sete virtudes da mulher de Provérbios 31 em sua rotina diária. Este vídeo oferece um plano prático para mulheres cristãs que desejam crescer em fé e sabedoria.
>  * **Por que é eficaz:** Apresenta um formato de desafio que incentiva o engajamento e oferece passos concretos para a aplicação dos princípios bíblicos.
> 
>  2. **Título:** Além do Avental: A Mulher de Provérbios 31 no Século XXI
>  * **Descrição:** Desmistificando a imagem tradicional da mulher de Provérbios 31, este vídeo explora como suas qualidades de empreendedora, administradora e mulher de fé se aplicam à vida moderna. Descubra como equilibrar carreira, família e chamado de Deus.
>  * **Por que é eficaz:** Aborda a relevância do texto para a realidade da mulher contemporânea, que muitas vezes se sente pressionada a corresponder a um ideal inatingível.
> 
>  3. **Título:** A Beleza que Floresce de Dentro: Descobrindo a Força em Provérbios 31
>  * **Descrição:** Este vídeo foca na beleza interior e na força de caráter da mulher de Provérbios 31, incentivando as espectadoras a cultivarem um relacionamento íntimo com Deus, que se reflete em suas ações e palavras.
>  * **Por que é eficaz:** Enfatiza que a verdadeira beleza emana do temor ao Senhor e da prática da sabedoria, valores que ressoam com o público cristão.
> 
>  4. **Título:** Provérbios 31 Para Solteiras: Lições de Sabedoria Para Todas as Fases da Vida
>  * **Descrição:** Muitas vezes, Provérbios 31 é interpretado como um manual para esposas. Este vídeo traz uma perspectiva diferente, mostrando como as solteiras também podem aplicar os princípios de sabedoria, diligência e serviço em suas vidas.
>  * **Por que é eficaz:** Amplia o alcance do texto, mostrando que seus ensinamentos são universais e relevantes para todas as mulheres, independentemente do estado civil.
> 
>  5.  **Título:** Marido à Sombra Dela: O Segredo Esquecido de Provérbios 31
>  *  **Descrição:** Explorando a importância do apoio e da parceria do marido para o florescimento da mulher virtuosa, este vídeo desafia os homens cristãos a encorajarem suas esposas em seus dons e talentos.
>  *  **Por que é eficaz:** Oferece uma perspectiva inovadora, destacando o papel fundamental do homem na vida da mulher de Provérbios 31, um aspecto muitas vezes negligenciado.
> 



🔍 Digite o número da ideia que você escolheu (ex: 1, 2, 3...):
  1. **Título:** Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa
  2. **Título:** Além do Avental: A Mulher de Provérbios 31 no Século XXI
  3. **Título:** A Beleza que Floresce de Dentro: Descobrindo a Força em Provérbios 31
  4. **Título:** Provérbios 31 Para Solteiras: Lições de Sabedoria Para Todas as Fases da Vida
  5.  **Título:** Marido à Sombra Dela: O Segredo Esquecido de Provérbios 31
> 1

✅ Ideia selecionada: 1. **Título:** Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa

📋 Configurando a geração de texto...
Tons disponíveis:
  1. Inspirador
  2. Informativo
  3. Pastoral
  4. Evangelístico
  5. Devocional
Escolha o número do tom (1-5): 1
Tamanhos disponíveis:
  1. Curto
  2. Médio
  3. Longo
Escolha o número do tamanho (1-3): 3

✨ Etapa 2: Gerando texto para a ideia selecionada...

📝 Gerando texto com tom 'inspirador' e tamanho 'longo'...


> Olá! Aqui está um roteiro completo para um vídeo cristão no YouTube, baseado na sua ideia. Espero que seja útil e inspirador!
> 
> 
> # Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa
> 
> ## Introdução: Descobrindo a Virtude em Provérbios 31
> 
> Olá, amados irmãos e irmãs em Cristo! Sejam muito bem-vindos a mais um vídeo inspirador em nosso canal. Hoje, vamos embarcar em uma jornada transformadora, explorando o livro de Provérbios, capítulo 31. Este texto sagrado nos apresenta um modelo de mulher virtuosa, cujas qualidades e ações são um farol para todos nós, homens e mulheres, em busca de uma vida que agrada a Deus.
> 
> Muitas vezes, Provérbios 31 é visto como um padrão inatingível, um ideal distante da realidade. No entanto, convido você a enxergar este capítulo como um guia prático, um desafio diário para cultivarmos virtudes em nosso caráter e impactarmos positivamente o mundo ao nosso redor. Durante os próximos 7 dias, vamos meditar em cada aspecto dessa mulher virtuosa, buscando aplicar seus princípios em nossa vida cotidiana. Preparados para este desafio? Então, vamos juntos trilhar este caminho de sabedoria e graça!
> 
> ## Dia 1: Temor do Senhor - A Base de Toda Virtude (Provérbios 31:30)
> 
> Nosso ponto de partida é o versículo 30: "Enganosa é a beleza e vã a formosura, mas a mulher que teme ao Senhor, essa será louvada". O temor do Senhor não é medo, mas sim um profundo respeito e reverência a Deus, reconhecendo Sua soberania e buscando agradá-Lo em tudo o que fazemos.
> 
> *   **Aplicação:** Comece o dia dedicando um tempo à oração e leitura da Bíblia. Peça a Deus para aumentar seu temor Dele e guiar seus passos.
> 
> ## Dia 2: Confiança e Fidelidade (Provérbios 31:11-12)
> 
> A mulher virtuosa é digna de confiança e fiel em seus compromissos. Seu marido confia nela de todo o coração, e ela sempre lhe faz o bem, e não o mal.
> 
> *   **Aplicação:** Analise seus relacionamentos. Você tem sido uma pessoa confiável e leal? Cumpra suas promessas e seja honesto em suas palavras e ações.
> 
> ## Dia 3: Diligência e Trabalho (Provérbios 31:13-19, 24, 27)
> 
> A mulher de Provérbios 31 é uma trabalhadora incansável, que cuida de sua casa, de sua família e ainda encontra tempo para negócios. Ela "busca lã e linho e trabalha de bom grado com as mãos" (Provérbios 31:13).
> 
> *   **Aplicação:** Dedique-se com afinco às suas tarefas, seja no trabalho, em casa ou na igreja. Lembre-se que o trabalho digno é uma forma de honrar a Deus.
> 
> ## Dia 4: Sabedoria e Planejamento (Provérbios 31:16, 18)
> 
> A mulher virtuosa não age impulsivamente. Ela "avalia um campo e o compra; com o que ganha planta uma vinha" (Provérbios 31:16). Ela é uma administradora sábia dos recursos que Deus lhe confia.
> 
> *   **Aplicação:** Antes de tomar decisões importantes, ore, peça conselho e planeje cuidadosamente. Evite gastos desnecessários e invista em coisas que trarão benefícios a longo prazo.
> 
> ## Dia 5: Generosidade e Compaixão (Provérbios 31:20)
> 
> A mulher virtuosa estende a mão aos pobres e ajuda os necessitados. "Abre a mão para o pobre e estende os braços aos necessitados" (Provérbios 31:20).
> 
> *   **Aplicação:** Procure oportunidades para ajudar pessoas em necessidade. Doe alimentos, roupas, tempo ou recursos financeiros. Lembre-se que "mais bem-aventurado é dar do que receber" (Atos 20:35).
> 
> ## Dia 6: Força e Dignidade (Provérbios 31:25)
> 
> A mulher virtuosa se reveste de força e dignidade, e enfrenta o futuro sem medo. "Reveste-se de força e dignidade; sorri diante do futuro" (Provérbios 31:25).
> 
> *   **Aplicação:** Confie em Deus para superar seus medos e ansiedades. Busque força Nele para enfrentar os desafios da vida com coragem e esperança.
> 
> ## Dia 7: Sabedoria em Suas Palavras (Provérbios 31:26)
> 
> A mulher virtuosa fala com sabedoria e ensina com amor. "Fala com sabedoria e ensina com amor" (Provérbios 31:26).
> 
> *   **Aplicação:** Use suas palavras para edificar, encorajar e abençoar as pessoas ao seu redor. Evite fofocas, críticas e palavras negativas.
> 
> ## Conclusão: Uma Vida de Virtude ao Alcance de Todos
> 
> Parabéns por completar o Desafio Provérbios 31! Ao longo desta semana, você mergulhou nas qualidades da mulher virtuosa e buscou aplicá-las em sua vida. Lembre-se que a virtude não é um destino, mas sim uma jornada contínua de crescimento e aperfeiçoamento.
> 
> Que este desafio seja apenas o começo de uma vida transformada pelo poder de Deus. Que você continue a buscar a sabedoria divina, a cultivar virtudes em seu caráter e a impactar o mundo com o amor de Cristo.
> 
> **Chamada para Ação:**
> 
> *   Compartilhe este vídeo com seus amigos e familiares.
> *   Deixe um comentário sobre qual virtude você mais precisa desenvolver.
> *   Inscreva-se no canal para receber mais conteúdos inspiradores.
> 
> ## Oração Final
> 
> Senhor, nós Te agradecemos por Tua Palavra, que nos ilumina e nos guia em direção a uma vida de virtude. Ajuda-nos a colocar em prática os ensinamentos de Provérbios 31, para que possamos Te honrar em tudo o que fazemos. Capacita-nos a ser pessoas confiáveis, diligentes, sábias, generosas, fortes e cheias de amor. Em nome de Jesus, amém!
> 



🔑 Extraindo palavras-chave para SEO...
Palavras-chave extraídas:
  • Provérbios 31
  • Mulher Virtuosa
  • Temor do Senhor
  • Sabedoria
  • Fidelidade
  • Diligência
  • Generosidade
  • Vida Cristã

✨ Etapa 3: Editando e otimizando o conteúdo...

Deseja adicionar mais palavras-chave para SEO? (s/n): s
Digite palavras-chave adicionais (separadas por vírgula): fé, oração, amor
✂️ Editando e otimizando o conteúdo para 'youtube'...

🎉 Conteúdo Final para YouTube 🎉

📹 Título do Vídeo: Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa

⏱️ Timestamps:
  • 00:00 - Introdução: Desvende o Código da Mulher Virtuosa (0:00)
  • 02:00 - Dia 1: Temor do Senhor - O Alicerce da Sabedoria (0:30)
  • 04:00 - Dia 2: Confiança Inabalável (1:30)
  • 06:00 - Dia 3: Mãos que Edificam (2:30)
  • 08:00 - Dia 4: Sabedoria nas Decisões (3:30)
  • 10:00 - Dia 5: Coração Generoso (4:30)
  • 12:00 - Dia 6: Revestida de Força (5:30)
  • 14:00 - Dia 7: Palavras que Curam (6:30)
  • 16:00 - Conclusão: A Virt

> Absolutamente! Aqui está uma versão revisada e otimizada do seu roteiro, pronta para o YouTube, juntamente com a descrição e as tags sugeridas:
> 
> 
> ```markdown
> # Desafio Provérbios 31: 7 Dias Para Uma Vida Mais Virtuosa
> 
> ## Introdução: Desvende o Código da Mulher Virtuosa (0:00)
> 
> Olá, família abençoada! Sejam muito bem-vindos ao nosso canal! Já se perguntou como viver uma vida que realmente agrada a Deus? Hoje, vamos desvendar os segredos de Provérbios 31 e descobrir como aplicar esses princípios transformadores no seu dia a dia! Prepare-se para 7 dias de desafios que vão mudar a sua vida!
> 
> Muitas vezes, Provérbios 31 parece um ideal impossível, mas quero te mostrar que é um guia prático para todos nós. Vamos juntos?
> 
> ## Dia 1: Temor do Senhor - O Alicerce da Sabedoria (0:30)
> 
> Nosso ponto de partida é Provérbios 31:30: "A beleza é enganosa, e a formosura é passageira; mas a mulher que teme ao Senhor será elogiada." O temor do Senhor é a base de toda virtude, um profundo respeito que guia nossas ações [1].
> 
> *   **Aplicação:** Comece o dia com oração e leitura da Palavra. Peça a Deus para encher seu coração com temor e sabedoria.
> 
> ## Dia 2: Confiança Inabalável (1:30)
> 
> A mulher virtuosa é sinônimo de confiança. Provérbios 31:11-12 diz: "O coração do seu marido confia nela, e não lhe falta ganho. Ela lhe faz bem e não mal, todos os dias da sua vida." [2]
> 
> *   **Aplicação:** Seja confiável em seus relacionamentos. Cumpra suas promessas e seja um exemplo de honestidade.
> 
> ## Dia 3: Mãos que Edificam (2:30)
> 
> A mulher de Provérbios 31 é uma força de trabalho! Provérbios 31:13-19, 24, 27 descreve sua dedicação em cuidar da família e prosperar nos negócios [2].
> 
> *   **Aplicação:** Dedique-se às suas tarefas com alegria e gratidão. Lembre-se de que o trabalho digno é uma forma de adorar a Deus.
> 
> ## Dia 4: Sabedoria nas Decisões (3:30)
> 
> A mulher virtuosa planeja com sabedoria. Provérbios 31:16 nos diz: "Ela avalia um campo e o compra; com o fruto do seu trabalho planta uma vinha." [2]
> 
> *   **Aplicação:** Ore e planeje antes de tomar decisões importantes. Busque conselho e evite impulsos.
> 
> ## Dia 5: Coração Generoso (4:30)
> 
> A compaixão transborda do coração da mulher virtuosa. Provérbios 31:20 diz: "Abre os braços ao pobre e estende as mãos ao necessitado." [2]
> 
> *   **Aplicação:** Seja sensível às necessidades ao seu redor. Doe seu tempo, recursos e amor ao próximo.
> 
> ## Dia 6: Revestida de Força (5:30)
> 
> A mulher virtuosa enfrenta o futuro com coragem. Provérbios 31:25 declara: "Reveste-se de força e dignidade; sorri diante do futuro." [2]
> 
> *   **Aplicação:** Confie em Deus para superar seus medos e ansiedades. Encontre força Nele para enfrentar os desafios.
> 
> ## Dia 7: Palavras que Curam (6:30)
> 
> A sabedoria guia suas palavras. Provérbios 31:26 nos ensina: "Fala com sabedoria e ensina com amor." [2]
> 
> *   **Aplicação:** Use suas palavras para edificar, encorajar e abençoar. Evite palavras que ferem e desmotivam.
> 
> ## Conclusão: A Virtude ao Seu Alcance (7:30)
> 
> Parabéns por completar o desafio! Lembre-se, a virtude é uma jornada, não um destino.
> 
> Que este desafio seja apenas o começo de uma vida transformada pelo poder de Deus. Que você continue a buscar a sabedoria divina, a cultivar virtudes em seu caráter e a impactar o mundo com o amor de Cristo.
> 
> **Chamada para Ação:**
> 
> *   Compartilhe este vídeo com seus amigos e familiares.
> *   Deixe um comentário sobre qual virtude você mais precisa desenvolver.
> *   Inscreva-se no canal para receber mais conteúdos inspiradores.
> 
> ## Oração Final (8:00)
> 
> Senhor, nós Te agradecemos por Tua Palavra, que nos ilumina e nos guia em direção a uma vida de virtude. Ajuda-nos a colocar em prática os ensinamentos de Provérbios 31, para que possamos Te honrar em tudo o que fazemos. Capacita-nos a ser pessoas confiáveis, diligentes, sábias, generosas, fortes e cheias de amor. Em nome de Jesus, amém!
> 
> ```
> 
> 
> **Descrição Otimizada para o YouTube:**
> 
> Descubra como Provérbios 31 pode transformar sua vida em 7 dias! Neste vídeo inspirador, exploramos os princípios da mulher virtuosa e como você pode aplicar cada um deles no seu dia a dia. Aprenda a cultivar o temor do Senhor, a ser confiável, diligente, sábio, generoso, forte e a usar suas palavras com amor. Prepare-se para um desafio que vai te aproximar de Deus e te tornar uma pessoa melhor! #Provérbios31 #MulherVirtuosa #VidaCristã #Fé #Oração #Sabedoria #Generosidade #Transformação #Desafio #Inspiração
> 
> **Tags Sugeridas:**
> 
> *   Provérbios 31
> *   Mulher Virtuosa
> *   Temor do Senhor
> *   Sabedoria
> *   Fidelidade
> *   Diligência
> *   Generosidade
> *   Vida Cristã
> *   Fé
> *   Oração
> *   Devocional
> *   Estudo Bíblico
> *   Mulheres na Bíblia
> *   Virtude
> *   Caráter Cristão
> *   Transformação Pessoal
> *   Inspiração Cristã
> 
> Deixe-me saber se você gostaria de mais alguma alteração!
> 



Deseja salvar o conteúdo gerado? (s/n): s
Digite um nome base para os arquivos (ex: mulher_proverbios31): "mulher_proverbios31"

✅ Conteúdo salvo como '"mulher_proverbios31"_roteiro.md' e '"mulher_proverbios31"_metadados.txt'
